In [3]:
import pandas as pd
import torch
import json
import numpy as np
import time
from code.bm25.bm25_utilities import BM25Utilities
from code.encoder.encoder_utilities import EncoderUtilities
from code.reciprocal_rank_fusion.reciprocal_rank_fusion_utilities import calculate_rrf_ranking
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient

c:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using GPU")
else:
    device = torch.device("cpu")
    print("Using CPU")

Using CPU


**Data**

In [5]:
test_df = pd.read_csv('rdr_test_qna.csv')
corpus_df = pd.read_csv('rdrsegmenter_legal_corpus.csv')

In [6]:
test_df.head()

,question_id,relevant_articles,segmented_question
0,8e2cfe626cebf209f94e0db8f147960c,['28/2020/nđ-cp%21'],mức xử_phạt đối_với hành_vi không tổ_chức khám...
1,0a32724630653580cc90c77bcf552baf,['64/2020/qh14%97'],tranh_chấp giữa cơ_quan ký_kết hợp_đồng dự_án ...
2,e66ae5eecc1672bac2c5799673666bda,['26/2019/nđ-cp%28'],"trình_tự cấp , cấp lại giấy chứng_nhận đủ điều..."
3,6b815a5b10736472101034aa8bd58146,['59/2020/qh14%180'],thành_viên hợp danh thì có được làm chủ doanh_...
4,4d4ed6ef19708f3e18fef48e4786a06e,['01/2016/qh14%12'],luật_sư có được miễn đào_tạo nghề đấu_giá hay ...


In [7]:
corpus_df.head()

,Unnamed: 0,index,article_title,content,law_article_id,processed_title,processed_splitted_content,segmented_title_content
0,0,0,Điều 1. Phạm vi áp dụng,"thông tư này hướng dẫn tuần tra, canh gác bảo ...","{'law_id': '01/2009/tt-bnn', 'article_id': '1'}",phạm vi áp dụng,phạm vi áp dụng: thông tư này hướng dẫn tuần t...,phạm_vi áp_dụng : thông_tư này hướng_dẫn tuần_...
1,1,1,Điều 2. Tổ chức lực lượng,"khoản 1. hàng năm trước mùa mưa, lũ, ủy ban n...","{'law_id': '01/2009/tt-bnn', 'article_id': '2'}",tổ chức lực lượng,"khoản 1. hàng năm trước mùa mưa, lũ, ủy ban n...","khoản 1 . hàng năm trước mùa mưa , lũ , uỷ_ban..."
2,2,2,Điều 2. Tổ chức lực lượng,"khoản 2. lực lượng tuần tra, canh gác đê được...","{'law_id': '01/2009/tt-bnn', 'article_id': '2'}",tổ chức lực lượng,tổ chức lực lượng: khoản 2. lực lượng tuần tr...,tổ_chức lực_lượng : khoản 2 . lực_lượng tuần_t...
3,3,3,Điều 2. Tổ chức lực lượng,"khoản 3. khi lũ, bão có diễn biến phức tạp, k...","{'law_id': '01/2009/tt-bnn', 'article_id': '2'}",tổ chức lực lượng,"tổ chức lực lượng: khoản 3. khi lũ, bão có di...","tổ_chức lực_lượng : khoản 3 . khi lũ , bão có ..."
4,4,4,Điều 3. Tiêu chuẩn của các thành viên thuộc lự...,"khoản 1. là người khoẻ mạnh, tháo vát, đủ khả...","{'law_id': '01/2009/tt-bnn', 'article_id': '3'}",tiêu chuẩn của các thành viên thuộc lực lượng ...,tiêu chuẩn của các thành viên thuộc lực lượng ...,tiêu_chuẩn của các thành_viên thuộc lực_lượng ...


In [8]:
print(len(test_df))
print(len(corpus_df))

640
391391


In [9]:
corpus = corpus_df['segmented_title_content'].tolist()

In [10]:
bm25_util = BM25Utilities(bm25_text=corpus)
rank = bm25_util.get_bm25_ranking(query=test_df['segmented_question'][0])

BM25Utilities initializing...


In [11]:
print(rank)

[ 32581 134103 385160 ... 124283 124284 195695]


In [29]:
class CorpusDataset:
    def __init__(self,
                 corpus_df, 
                 url = 'https://4d31c99a-9390-4174-8547-167526f0138b.europe-west3-0.gcp.cloud.qdrant.io:6333',
                 api_key = 'WcOwiXDWcFcKJBXkI2zH9LHQ2he0npQNkYTEmS84UGx4kcLwRbMVKg'):
        self.url =url
        self.api_key = api_key
        self.collection_name = 'embedding_legal_1'
        self.corpus_df = corpus_df
        self.client = QdrantClient(url=self.url, api_key=self.api_key)
        self.model = SentenceTransformer('bkai-foundation-models/vietnamese-bi-encoder')

    def encode(self, text):
        return self.model.encode([text])[0]
    
    def query(self, segmented_question, topk=10):
        wait = 0.1
        while True:
            try:
                results = self.client.search(
                            collection_name = self.collection_name,
                            query_vector=self.encode(segmented_question),
                            limit=topk,
                        )
                break
            except:
                time.sleep(wait)
                wait*=2
        
        content_results = {}
        for point in results:
            law_id = point.payload['law_article_id']['law_id']
            article_id = point.payload['law_article_id']['article_id']
            key = law_id + '%' + article_id
            if key not in content_results:
                content_results[key] = [point.id]
            else:  
                content_results[key].append(point.id)
        return content_results
    
    def get_id(self, point_id):
        law_article_id = self.corpus_df.iloc[point_id]['law_article_id']
        law_article_id = eval(law_article_id)
        return law_article_id['law_id'] +'%'+ law_article_id['article_id']

In [30]:
corpus_dataset = CorpusDataset(corpus_df=corpus_df)
print(corpus_dataset.get_id(1))

01/2009/tt-bnn%2


In [38]:
class HybridSearch:
    def __init__(self, query_df, corpus_df, topk=[1,10,100]):
        self.query_df = query_df
        self.corpus_df = corpus_df
        self.corpus =  corpus_df['segmented_title_content'].tolist()
        self.topk = topk
        self.bm25_util = BM25Utilities(bm25_text=self.corpus)
        self.corpus_util = CorpusDataset(self.corpus_df)

    def get_bm25_rank(self, query, topk):
        return self.bm25_util.get_bm25_ranking(query=query)[:topk]
    
    def get_cosine_rank(self, query, topk):
        rank = self.corpus_util.query(query, topk=topk)
        res = []
        for i in rank.values():
            res.extend(i)
        return np.array(res)
    
    def get_rrf_rank(self, query, topk):
        bm25_rank = self.get_bm25_rank(query, topk)
        cosine_rank = self.get_cosine_rank(query, topk)
        res = calculate_rrf_ranking(bm25_rank, cosine_rank)
        return res
    
    def get_rerank(self, rrf_rank, bm25_rank):
        rrf_rank = [int(i) for i in rrf_rank]
        rerank = bm25_rank[rrf_rank][::-1]
        return rerank
    
    def get_law_article_id(self, rerank):
        res = []
        for point_id in rerank:
            law_article_id = self.corpus_util.get_id(point_id)
            if law_article_id not in res:
                res.append(law_article_id)
        return res
        
    def test(self):
        for k in self.topk:
            res={}
            for i in range(len(self.query_df)):
                query = self.query_df['segmented_question'][i]
                query_id  = self.query_df['question_id'][i]
                bm25_rank = self.get_bm25_rank(query, k)
                cosine_rank = self.get_cosine_rank(query, k)
                rrf_rank = self.get_rrf_rank(query, k)
                rerank = self.get_rerank(rrf_rank, bm25_rank)
                law_article_id = self.get_law_article_id(rerank)
                res[query_id] = law_article_id
            with open('result_'+str(k)+'.json', 'w') as f:
                json.dump(res, f)

    

In [37]:
hybrid = HybridSearch(test_df['segmented_question'], corpus_df=corpus_df)
rrf_rank = hybrid.get_rrf_rank(test_df['segmented_question'][0], topk=10)
rerank = hybrid.get_rerank(rrf_rank, hybrid.get_bm25_rank(test_df['segmented_question'][0]))
print(hybrid.get_law_article_id(rerank))

BM25Utilities initializing...
['71/2019/nđ-cp%65', '96/2020/nđ-cp%12', '05/2019/tt-btc%80', '128/2020/nđ-cp%32', '1498/2005/qđ-nhnn%2', '155/2018/nđ-cp%143', '239/2009/ttlt-btc-vksndtc-tandtc%1', '06/2020/tt-nhnn%3']


[ 32581 134103 385160  32580  32579 145387 210248 107991  38825 342828]
